In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [95]:
def mort_clean_for_king(data):
    mort = pd.read_csv(data)
    mort.Year_Month = pd.to_datetime(mort.Year_Month)
    mort.rename(columns = {'30-Fixed': '30FixedMortgage'}, inplace = True)
    mort.iloc[:, :-1].to_csv('mortgage_for_king.csv', index = False)
    return (mort)

In [99]:
def mort_clean_for_king_and_aus(data):
    mort = mort_clean_for_king(data)
    mort['30FixedMortgage_3MoMedian'] = mort['30FixedMortgage'].rolling(3).median()[2:]
    mort['30FixedMortgage_3MoMedian'] = mort['30FixedMortgage_3MoMedian'].shift(-2)
    mort.loc[mort.index % 3 == 0, ['Year_Month', '30FixedMortgage_3MoMedian']].to_csv('mortgage_for_austin.csv', index = False)
    return (mort)

In [100]:
mort = mort_clean_for_king_and_aus('mortgage_data_v0404.csv')

In [83]:
def stock_clean_for_king(data, stock):
    df = pd.read_csv(data)
    df.Date = pd.to_datetime(df.Date)
    df.iloc[:, [0, 4]].to_csv('stock_{}_for_king.csv'.format(stock), index = False)

In [87]:
def stock_clean_for_aus(data, stock):
    df = pd.read_csv(data)
    df.Date = pd.to_datetime(df.Date)
    df = df.groupby(df['Date'].dt.to_period('Q'))['Close'].median().to_frame()
    mort_date = mort.loc[mort.index % 3 == 0, ['Year_Month']]
    df['Date_3Mo'] = mort_date.iloc[:-4, :]['Year_Month'].to_list()
    df.reset_index(inplace = True)
    df.loc[:, ['Date_3Mo', 'Close']].to_csv('stock_{}_for_austin.csv'.format(stock), index = False)

In [90]:
def stock_clean(stock):
    stock_clean_for_king('{}_0328.csv'.format(stock), stock)
    stock_clean_for_aus('{}_0328.csv'.format(stock), stock)

In [92]:
for stock in ['SPY', 'QQQ', 'DGL']:
    stock_clean(stock)